# Data Engineering

In [175]:
import pandas as pd
import numpy as np
import re

import fundamentus

from google.cloud import storage
import os
import requests
import requests_cache
import logging
import time
from datetime import date

In [2]:
credentials_path = 'datascience-capstone-project-05b1642f45c3.json'

## 1 - Data Collection

For the data gathering, two repositories on Github were combined, so that the historical data could be scraped from the web. This Raw data is beeing stored in the cloud (GCP) and are the main data used for the project.

Repositories utilized:
* https://github.com/mv/fundamentus-api
* https://github.com/Victorcorcos/bovespa-winner

In [3]:
def perc_to_float(val):
    """
    Percent to float
      - replace string in pt-br to float
      - from '45,56%' to 0.4556
    Input:
        (DataFrame, column_name)
    """

    res = val
    res = res.replace( to_replace=r'[%]', value='' , regex=True )
    res = res.replace( to_replace=r'[.]', value='' , regex=True )
    res = res.replace( to_replace=r'[,]', value='.', regex=True )
    res = res.astype(float) / 100

    return res

def _rename_cols(data):
    """
    Rename columns in DataFrame
      - use a valid Python identifier
      - so each column can be a DataFrame property
      - Example:
          df.pl > 0
    """

    df2 = pd.DataFrame()

    ## Fix: rename columns
    df2['cotacao'  ] = data['Cotação'          ]
    df2['pl'       ] = data['P/L'              ]
    df2['pvp'      ] = data['P/VP'             ]
    df2['psr'      ] = data['PSR'              ]
    df2['dy'       ] = data['Div.Yield'        ]
    df2['pa'       ] = data['P/Ativo'          ]
    df2['pcg'      ] = data['P/Cap.Giro'       ]
    df2['pebit'    ] = data['P/EBIT'           ]
    df2['pacl'     ] = data['P/Ativ Circ.Liq'  ]
    df2['evebit'   ] = data['EV/EBIT'          ]
    # df2['evebitda' ] = data['EV/EBITDA'        ]
    df2['mrgebit'  ] = data['Mrg Ebit'         ]
    df2['mrgliq'   ] = data['Mrg. Líq.'        ]
    df2['roic'     ] = data['ROIC'             ]
    df2['roe'      ] = data['ROE'              ]
    df2['liqc'     ] = data['Liq. Corr.'       ]
    df2['liq2m'    ] = data['Liq.2meses'       ]
    df2['patrliq'  ] = data['Patrim. Líq'      ]
    df2['divbpatr' ] = data['Dív.Brut/ Patrim.']
    df2['c5y'      ] = data['Cresc. Rec.5a'    ]

    return df2

def get_resultado_raw(url):
    """
    Get data from fundamentus:
      URL:
        http://fundamentus.com.br/resultado.php
    RAW:
      DataFrame preserves original HTML header names
    Output:
      DataFrame
    """

    ##
    ## Busca avançada por empresa
    ##
    # url = 'http://www.fundamentus.com.br/resultado.php'
    hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01',
           'Accept-Encoding': 'gzip, deflate',
           }

    with requests_cache.enabled():
        content = requests.get(url, headers=hdr)

        if content.from_cache:
            logging.debug('.../resultado.php: [CACHED]')
        else: # pragma: no cover
            logging.debug('.../resultado.php: sleeping...')
            time.sleep(.500) # 500 ms


    ## parse + load
    df = pd.read_html(content.text, decimal=",", thousands='.')[0]

    ## Fix: percent string
    df['Div.Yield']     = perc_to_float( df['Div.Yield']     )
    df['Mrg Ebit']      = perc_to_float( df['Mrg Ebit']      )
    df['Mrg. Líq.']     = perc_to_float( df['Mrg. Líq.']     )
    df['ROIC']          = perc_to_float( df['ROIC']          )
    df['ROE']           = perc_to_float( df['ROE']           )
    df['Cresc. Rec.5a'] = perc_to_float( df['Cresc. Rec.5a'] )

    ## index by 'Papel', instead of 'int'
    df.index = df['Papel']
    df.drop('Papel', axis='columns', inplace=True)
    df.sort_index(inplace=True)

    ## naming
    df.name = 'Fundamentus: HTML names'
    df.columns.name = 'Multiples'
    df.index.name = 'papel'

    ## return sorted by 'papel'
    return df


def get_resultado(url):
    """
    Data from fundamentus, fixing header names.
      URL:
        given from the user
      Obs:
        DataFrame uses short header names
    Output:
      DataFrame
    """

    ## get RAW data
    data1 = get_resultado_raw(url)

    ## rename!
    data2 = _rename_cols(data1)

    ## metadata
    data2.name = 'Fundamentus: short names'
    data2.columns.name = 'Multiples'
    data2.index.name = 'papel'

    ## remove duplicates
#   df = data2.drop_duplicates(subset=['cotacao','pl','pvp'], keep='last')
    df = data2.drop_duplicates(keep='first')

    return df


In [4]:
def initialize_bucket(credentials_path, create_bucket=False):

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

    client = storage.Client()
    bucket = client.bucket('storage-barsianize')
    
    if create_bucket:
        bucket.location = 'US-EAST1'
        bucket.create()

    return client, bucket
    

In [5]:
def get_windows(max, min, window=5, step=1):

    maxes = list(range(min+window,max+1, step))
    mins = list(range(min, max+1-window, step))
    sequences = []
    if len(maxes)==len(mins):
        for i in range(len(maxes)):
            sequence = list(range(mins[i],maxes[i]+1,step))
            sequences.append(sequence)
        return sequences
    else:
        print(f'error!!! min {maxes}, max {maxes}')

### Get historical data

In [195]:
urls = {
    2008: 'https://web.archive.org/web/20080613050801/http://www.fundamentus.com.br/resultado.php',
    2009: 'https://web.archive.org/web/20090123022224/http://www.fundamentus.com.br/resultado.php',
    2010: 'https://web.archive.org/web/20100115191626/http://www.fundamentus.com.br/resultado.php',
    2011: 'https://web.archive.org/web/20110113192117/http://www.fundamentus.com.br/resultado.php',
    2012: 'https://web.archive.org/web/20120106023830/http://www.fundamentus.com.br/resultado.php',
    2013: 'https://web.archive.org/web/20130105004012/http://www.fundamentus.com.br/resultado.php',
    2014: 'https://web.archive.org/web/20140108164618/http://www.fundamentus.com.br/resultado.php',
    2015: 'https://web.archive.org/web/20150119231047/http://www.fundamentus.com.br/resultado.php',
    2016: 'https://web.archive.org/web/20160106101916/http://www.fundamentus.com.br/resultado.php',
    2017: 'https://web.archive.org/web/20170505164235/http://www.fundamentus.com.br/resultado.php',
    2018: 'https://web.archive.org/web/20180105120409/http://www.fundamentus.com.br/resultado.php',
    2019: 'https://web.archive.org/web/20190102202956/http://www.fundamentus.com.br/resultado.php',
    2020: 'https://web.archive.org/web/20200122200313/http://www.fundamentus.com.br/resultado.php',
    2021: 'https://web.archive.org/web/20210227034423/http://www.fundamentus.com.br/resultado.php',
    2022: 'https://web.archive.org/web/20220314021607/http://www.fundamentus.com.br/resultado.php',
    2023: 'https://web.archive.org/web/20230324145930/http://www.fundamentus.com.br/resultado.php'
  }
  
years = list(range(2008,2024))

def get_dates(urls):
    dict_dates = {}
    for key, value in urls.items():
        str_1 = re.split('/web/',value)[1]
        str_2 = re.split('/http',str_1)[0]
        str_date = f'{str_2[:4]}-{str_2[4:6]}-{str_2[6:8]}'
        dict_dates[int(str_2[:4])] = pd.to_datetime(str_date)
    return dict_dates

dates = get_dates(urls)

df_full = pd.DataFrame()
for year in years:
    df = get_resultado(urls[year])
    df['year'] = year
    df['date'] = dates[year]
    df_full = pd.concat([df_full,df])
    
df_full = df_full.reset_index()

In [196]:
df_full.head()

Multiples,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,...,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year,date
0,ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,...,0.0000,0.0000,0.0000,0.00,0.0,1.107010e+09,0.00,0.0000,2008,2008-06-13
1,ABCB4,9.00,0.00,1.10,0.000,0.0489,0.000,0.00,0.00,0.00,...,0.0000,0.0000,0.0000,0.00,4160460.0,1.107010e+09,0.00,0.0000,2008,2008-06-13
2,ABNB3,17.35,12.39,3.27,1.845,0.0254,2.590,7.26,8.45,5.04,...,0.1490,0.3465,0.2639,3.28,2586090.0,2.733490e+08,0.00,0.1446,2008,2008-06-13
3,ABYA3,16.00,25.67,4.13,4.399,0.0079,0.570,2.10,17.45,2.51,...,0.1714,0.0356,0.1608,1.94,5915230.0,1.971610e+08,2.40,0.0000,2008,2008-06-13
4,ACES3,95.27,8.79,2.28,1.693,0.0000,1.438,6.40,7.20,3.77,...,0.1927,0.2690,0.2595,2.05,181013.0,3.105800e+09,0.09,0.1334,2008,2008-06-13


### Get detailed information about each ticker

In [197]:
papeis = df_full['papel'].unique()

df_papeis = pd.DataFrame()
for papel in papeis:
    try:
        df = fundamentus.get_papel(papel)
        df_papeis = pd.concat([df_papeis,df])
    except:
        print(f'fail papel {papel}')

2023-03-25 13:13:39,604 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:39,668 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:39,740 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:39,760 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:39,827 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:39,890 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:39,953 [detalhes.get_papel] INFO: detalhes: call: get..._papel()


fail papel ABNB3


2023-03-25 13:13:40,015 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,081 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,145 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,208 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,271 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,337 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,356 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,420 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,483 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,546 [detalhes.get_papel] INFO: detalhes: call: get..._papel()


fail papel AGEN11


2023-03-25 13:13:40,610 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,673 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,738 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,805 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,868 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,931 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:40,993 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:41,055 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:41,117 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:41,179 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:41,241 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13:41,305 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:13

...

2023-03-25 13:14:30,800 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:30,866 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:30,931 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:30,995 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,058 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,122 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,131 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,194 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,256 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,320 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,385 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,449 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14

fail papel OGSA3


2023-03-25 13:14:31,696 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,760 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,824 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,888 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:31,952 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:32,018 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:32,083 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:32,148 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:32,211 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:32,274 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:32,338 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:32,401 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14

fail papel LIQO3


2023-03-25 13:14:34,642 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:34,707 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:34,770 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:34,835 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:34,897 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:34,960 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:34,971 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:35,039 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:35,102 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:35,167 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:35,231 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14:35,295 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-03-25 13:14

In [198]:
df_full[df_full['papel']=='ITUB4']

Multiples,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,...,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year,date
1885,ITUB4,37.91,19.86,3.55,0.0,0.0188,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1786,0.0,1.686560e+08,4.886160e+10,0.0,0.1767,2010,2010-01-15
2654,ITUB4,40.25,14.55,3.22,0.0,0.0246,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2210,0.0,2.156390e+08,5.722510e+10,0.0,0.2734,2011,2011-01-13
3440,ITUB4,34.66,13.27,2.09,0.0,0.0292,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1572,0.0,2.463310e+08,7.591600e+10,0.0,-0.4082,2012,2012-01-06
4236,ITUB4,35.51,14.42,1.93,0.0,0.0325,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1337,0.0,2.728300e+08,8.416040e+10,0.0,-0.4905,2013,2013-01-05
5051,ITUB4,31.27,14.83,1.80,0.0,0.0316,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1214,0.0,3.127270e+08,8.733480e+10,0.0,-0.4519,2014,2014-01-08
5870,ITUB4,33.10,12.07,1.88,0.0,0.0299,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1559,0.0,4.247690e+08,9.726990e+10,0.0,-0.1391,2015,2015-01-19
6689,ITUB4,25.34,6.96,1.43,0.0,0.0584,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2050,0.0,4.617880e+08,1.080200e+11,0.0,0.2599,2016,2016-01-06
7524,ITUB4,38.10,12.31,2.13,0.0,0.0468,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1733,0.0,4.294830e+08,1.175570e+11,0.0,0.1019,2017,2017-05-05
8372,ITUB4,45.10,13.70,2.35,0.0,0.0334,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1717,0.0,4.841310e+08,1.255750e+11,0.0,0.0391,2018,2018-01-05
9224,ITUB4,35.50,17.59,2.77,0.0,0.0595,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1576,0.0,7.078580e+08,1.255340e+11,0.0,-0.1029,2019,2019-01-02


In [199]:
df_papeis.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Cotacao,Data_ult_cot,Min_52_sem,Max_52_sem,Vol_med_2m,...,Rec_Servicos_3m,Lucro_Liquido_3m,Disponibilidades,Ativo_Circulante,Div_Bruta,Div_Liquida,Receita_Liquida_12m,EBIT_12m,Receita_Liquida_3m,EBIT_3m
ABCB3,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,0.00,1899-12-30,0.00,0.00,0,...,118020000,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABCB4,ABCB4,PN N2,ABC Brasil PN N2,Intermediários Financeiros,Bancos,17.91,2023-01-04,14.06,22.30,13880200,...,118020000,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABYA3,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,4.91,2010-02-11,0.00,0.00,0,...,NaN,-766000,29556000,498222000,383159000,353603000,250657000,25802000,67990000,18057000
ACES3,ACES3,ON,ARCELORMITTAL INOX BRASIL ON,Siderurgia e Metalurgia,Siderurgia,95.27,2008-04-18,0.00,0.00,0,...,NaN,163789000,922471000,2157470000,279166000,-643305000,4182420000,983634000,949114000,174503000
ACES4,ACES4,PN,ARCELORMITTAL INOX BRASIL PN,Siderurgia e Metalurgia,Siderurgia,94.73,2008-04-24,0.00,0.00,0,...,NaN,163789000,922471000,2157470000,279166000,-643305000,4182420000,983634000,949114000,174503000


In [200]:
df_full.head()

Multiples,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,...,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year,date
0,ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,...,0.0000,0.0000,0.0000,0.00,0.0,1.107010e+09,0.00,0.0000,2008,2008-06-13
1,ABCB4,9.00,0.00,1.10,0.000,0.0489,0.000,0.00,0.00,0.00,...,0.0000,0.0000,0.0000,0.00,4160460.0,1.107010e+09,0.00,0.0000,2008,2008-06-13
2,ABNB3,17.35,12.39,3.27,1.845,0.0254,2.590,7.26,8.45,5.04,...,0.1490,0.3465,0.2639,3.28,2586090.0,2.733490e+08,0.00,0.1446,2008,2008-06-13
3,ABYA3,16.00,25.67,4.13,4.399,0.0079,0.570,2.10,17.45,2.51,...,0.1714,0.0356,0.1608,1.94,5915230.0,1.971610e+08,2.40,0.0000,2008,2008-06-13
4,ACES3,95.27,8.79,2.28,1.693,0.0000,1.438,6.40,7.20,3.77,...,0.1927,0.2690,0.2595,2.05,181013.0,3.105800e+09,0.09,0.1334,2008,2008-06-13


### Storing the raw data into Google Cloud

In [201]:
client, bucket = initialize_bucket(credentials_path)

# upload raw fundamentalist data to Google Cloud Storage
blob = bucket.blob('raw/df_full.csv')
blob.upload_from_string(df_full.to_csv(), 'text/csv')
# upload raw fundamentalist data to Google Cloud Storage
blob = bucket.blob('raw/df_papeis.csv')
blob.upload_from_string(df_papeis.to_csv(), 'text/csv')
# upload raw fundamentalist data to Google Cloud Storage
# blob = bucket.blob('raw/categories.csv')
# blob.upload_from_string(categories.to_csv(), 'text/csv')

## 2 - Data Cleansing

### Load data

In [202]:
client, bucket = initialize_bucket(credentials_path)

path = "gs://storage-barsianize/raw/df_full.csv"
df_full =  pd.read_csv(path, index_col=0)

path = "gs://storage-barsianize/raw/df_papeis.csv"
df_papeis = pd.read_csv(path, index_col=0)

Desired information about each ticker

In [203]:
df_papeis.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Cotacao,Data_ult_cot,Min_52_sem,Max_52_sem,Vol_med_2m,...,Rec_Servicos_3m,Lucro_Liquido_3m,Disponibilidades,Ativo_Circulante,Div_Bruta,Div_Liquida,Receita_Liquida_12m,EBIT_12m,Receita_Liquida_3m,EBIT_3m
ABCB3,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,0.00,1899-12-30,0.00,0.0,0,...,118020000.0,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABCB4,ABCB4,PN N2,ABC Brasil PN N2,Intermediários Financeiros,Bancos,17.91,2023-01-04,14.06,22.3,13880200,...,118020000.0,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABYA3,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,4.91,2010-02-11,0.00,0.0,0,...,NaN,-766000,29556000.0,4.982220e+08,383159000.0,353603000.0,2.506570e+08,25802000.0,67990000.0,18057000.0
ACES3,ACES3,ON,ARCELORMITTAL INOX BRASIL ON,Siderurgia e Metalurgia,Siderurgia,95.27,2008-04-18,0.00,0.0,0,...,NaN,163789000,922471000.0,2.157470e+09,279166000.0,-643305000.0,4.182420e+09,983634000.0,949114000.0,174503000.0
ACES4,ACES4,PN,ARCELORMITTAL INOX BRASIL PN,Siderurgia e Metalurgia,Siderurgia,94.73,2008-04-24,0.00,0.0,0,...,NaN,163789000,922471000.0,2.157470e+09,279166000.0,-643305000.0,4.182420e+09,983634000.0,949114000.0,174503000.0


In [82]:
df_papeis.columns

Index(['Papel', 'Tipo', 'Empresa', 'Setor', 'Subsetor', 'Cotacao',
       'Data_ult_cot', 'Min_52_sem', 'Max_52_sem', 'Vol_med_2m',
       'Valor_de_mercado', 'Valor_da_firma', 'Ult_balanco_processado',
       'Nro_Acoes', 'PL', 'PVP', 'PEBIT', 'PSR', 'PAtivos', 'PCap_Giro',
       'PAtiv_Circ_Liq', 'Div_Yield', 'EV_EBITDA', 'EV_EBIT', 'Cres_Rec_5a',
       'LPA', 'VPA', 'Marg_Bruta', 'Marg_EBIT', 'Marg_Liquida', 'EBIT_Ativo',
       'ROIC', 'ROE', 'Liquidez_Corr', 'Div_Br_Patrim', 'Giro_Ativos', 'Ativo',
       'Cart_de_Credito', 'Depositos', 'Patrim_Liq', 'Result_Int_Financ_12m',
       'Rec_Servicos_12m', 'Lucro_Liquido_12m', 'Result_Int_Financ_3m',
       'Rec_Servicos_3m', 'Lucro_Liquido_3m', 'Disponibilidades',
       'Ativo_Circulante', 'Div_Bruta', 'Div_Liquida', 'Receita_Liquida_12m',
       'EBIT_12m', 'Receita_Liquida_3m', 'EBIT_3m'],
      dtype='object')

In [219]:
# info_papeis = ['Papel','Tipo', 'Empresa', 'Setor', 'Subsetor','Data_ult_cot']
info_papeis = ['Papel','Tipo', 'Empresa', 'Setor', 'Subsetor','Data_ult_cot','Lucro_Liquido_3m',\
               'Disponibilidades','Ativo_Circulante','Div_Bruta','Div_Liquida','Receita_Liquida_12m',\
               'EBIT_12m','Receita_Liquida_3m','EBIT_3m']
               
df_papeis_clean = df_papeis[info_papeis]

df_papeis_clean = df_papeis_clean.dropna()
df_papeis_clean = df_papeis_clean.drop_duplicates()

In [221]:
df_papeis_clean['Data_ult_cot'] = pd.to_datetime(df_papeis_clean['Data_ult_cot'])

In [222]:
df_completed = df_papeis_clean.merge(df_full, how='left', left_index=True, right_on='papel').drop_duplicates()

In [225]:
df_completed.isna().mean().sort_values(ascending=False)

Papel                  0.0
psr                    0.0
pa                     0.0
pcg                    0.0
pebit                  0.0
pacl                   0.0
evebit                 0.0
mrgebit                0.0
mrgliq                 0.0
roic                   0.0
roe                    0.0
liqc                   0.0
liq2m                  0.0
patrliq                0.0
divbpatr               0.0
c5y                    0.0
year                   0.0
dy                     0.0
pvp                    0.0
Tipo                   0.0
pl                     0.0
Empresa                0.0
Setor                  0.0
Subsetor               0.0
Data_ult_cot           0.0
Lucro_Liquido_3m       0.0
Disponibilidades       0.0
Ativo_Circulante       0.0
Div_Bruta              0.0
Div_Liquida            0.0
Receita_Liquida_12m    0.0
EBIT_12m               0.0
Receita_Liquida_3m     0.0
EBIT_3m                0.0
papel                  0.0
cotacao                0.0
date                   0.0
d

In [226]:
client, bucket = initialize_bucket(credentials_path)

# upload raw fundamentalist data to Google Cloud Storage
blob = bucket.blob('trusted/df_completed.csv')
blob.upload_from_string(df_completed.to_csv(), 'text/csv',)

## 3 - Data Preparation

In [227]:
client, bucket = initialize_bucket(credentials_path)

In [237]:
path = "gs://storage-barsianize/trusted/df_completed.csv"
df_completed =  pd.read_csv(path, index_col=0)
df_completed['Data_ult_cot'] = pd.to_datetime(df_completed['Data_ult_cot'])

In [238]:
df_completed.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Data_ult_cot,Lucro_Liquido_3m,Disponibilidades,Ativo_Circulante,Div_Bruta,...,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year,date
3,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,2010-02-11,-766000,29556000.0,498222000.0,383159000.0,...,0.1714,0.0356,0.1608,1.94,5915230.0,197161000.0,2.40,0.0000,2008,2008-06-13
739,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,2010-02-11,-766000,29556000.0,498222000.0,383159000.0,...,0.5366,0.0376,0.4811,1.05,396462.0,332903000.0,1.34,0.0000,2009,2009-01-23
1481,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,2010-02-11,-766000,29556000.0,498222000.0,383159000.0,...,-0.5939,-0.0997,-0.3447,1.31,7905540.0,299602000.0,1.23,-0.4169,2010,2010-01-15
2239,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,2010-02-11,-766000,29556000.0,498222000.0,383159000.0,...,-0.0096,0.0278,-0.0082,2.09,0.0,292060000.0,1.31,0.1641,2011,2011-01-13
3023,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,2010-02-11,-766000,29556000.0,498222000.0,383159000.0,...,-0.0096,0.0278,-0.0082,2.09,0.0,292060000.0,1.31,0.1641,2012,2012-01-06


In [245]:
df_completed.columns

Index(['Papel', 'Tipo', 'Empresa', 'Setor', 'Subsetor', 'Data_ult_cot',
       'Lucro_Liquido_3m', 'Disponibilidades', 'Ativo_Circulante', 'Div_Bruta',
       'Div_Liquida', 'Receita_Liquida_12m', 'EBIT_12m', 'Receita_Liquida_3m',
       'EBIT_3m', 'papel', 'cotacao', 'pl', 'pvp', 'psr', 'dy', 'pa', 'pcg',
       'pebit', 'pacl', 'evebit', 'mrgebit', 'mrgliq', 'roic', 'roe', 'liqc',
       'liq2m', 'patrliq', 'divbpatr', 'c5y', 'year', 'date',
       'Data_ult_cot_year'],
      dtype='object')

In [240]:
df_completed['Data_ult_cot_year'] = df_completed['Data_ult_cot'].dt.year

In [241]:
df_actual = df_completed[df_completed['Data_ult_cot_year'] == pd.to_datetime(date.today()).year]

In [242]:
tickers = df_actual['Papel'].unique()

In [243]:
tickers

array(['AGRO3', 'ALPA3', 'ALPA4', 'AZEV3', 'AZEV4', 'BAHI3', 'BALM3',
       'BALM4', 'BDLL4', 'BEEF3', 'BIOM3', 'BMKS3', 'BOBR4', 'BRAP3',
       'BRAP4', 'BRGE11', 'BRGE3', 'BRKM3', 'BRKM5', 'BRML3', 'CALI3',
       'CAMB3', 'CBEE3', 'CCRO3', 'CEBR3', 'CEBR5', 'CEBR6', 'CEDO4',
       'CEEB3', 'CGAS3', 'CGAS5', 'CGRA3', 'CGRA4', 'CLSC3', 'CMIG3',
       'CMIG4', 'COCE5', 'CPFE3', 'CPLE3', 'CPLE6', 'CSAB3', 'CSAB4',
       'CSAN3', 'CSMG3', 'CSNA3', 'CSRN3', 'CSRN5', 'CSRN6', 'CTKA4',
       'CTNM4', 'CTSA3', 'CTSA4', 'CYRE3', 'DASA3', 'DOHL4', 'EALT3',
       'EALT4', 'EKTR3', 'EKTR4', 'ELET3', 'ELET6', 'EMAE4', 'EMBR3',
       'ENBR3', 'ENGI3', 'ENGI4', 'EQTL3', 'ESTR4', 'ETER3', 'EUCA3',
       'EUCA4', 'EVEN3', 'EZTC3', 'FESA3', 'FESA4', 'FHER3', 'GEPA3',
       'GEPA4', 'GFSA3', 'GGBR3', 'GGBR4', 'GOAU3', 'GOAU4', 'GOLL4',
       'GRND3', 'GSHP3', 'GUAR3', 'HAGA3', 'HBOR3', 'HETA4', 'HOOT4',
       'HYPE3', 'IGBR3', 'INEP3', 'INEP4', 'ITSA3', 'ITSA4', 'JBSS3',
       'JFEN3', 'JH

In [246]:
stock_info = ['Papel','Tipo','Empresa','Setor','Subsetor']

indicators = ['cotacao', 'pl', 'pvp', 'psr', 'dy', 'pa', 'pcg', 'pebit',
              'pacl', 'evebit', 'mrgebit', 'mrgliq', 'roic', 'roe', 'liqc', 'liq2m',
              'patrliq', 'divbpatr', 'c5y', 'Lucro_Liquido_3m', 'Disponibilidades', 'Ativo_Circulante', 'Div_Bruta',
              'Div_Liquida', 'Receita_Liquida_12m', 'EBIT_12m', 'Receita_Liquida_3m',
              'EBIT_3m','date','year']

In [247]:
df_indicators = df_actual[np.isin(df_actual['Papel'],tickers)][stock_info + indicators]

In [248]:
data = df_actual[df_actual['Papel']==tickers[2]].sort_values('year', ascending=False)

max = data['year'].max()
min = data['year'].min()
windows = get_windows(max, min, window=5, step=1)

In [249]:
def get_variations(data, windows, stock_info_cols):
    
    data_line_full = pd.DataFrame()
    cols = ['cotacao','pl','pvp','dy','roe','patrliq','divbpatr','Div_Bruta',
              'Div_Liquida', 'Receita_Liquida_12m','EBIT_12m', 'Receita_Liquida_3m',
              'EBIT_3m','Lucro_Liquido_3m','Ativo_Circulante']
    extra_cols = list(data.columns[~np.isin(data.columns, stock_info_cols + cols)])
    
    for window in windows:
        window.sort(reverse=True)

        data_window = data[np.isin(data['year'],window)]
        data_window = data_window.sort_values(by='year', ascending=False)

        data_line = {}

        for col in cols:
            data_line[col] = data_window[col].iloc[0]
            try:
                data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
            except:
                data_line[f'{col}_rel'] = 0

            data_line[f'{col}_median'] = data_window[col].median()
         
            try:
                data_line[f'{col}_median_rel'] = data_window[col].median()/ data_window[col].iloc[-1]
            except:
                data_line[f'{col}_median_rel'] = 0

            data_line[f'{col}_mean'] = data_window[col].mean()

            try:
                data_line[f'{col}_mean_rel'] = data_window[col].mean()/ data_window[col].iloc[-1]
            except:
                data_line[f'{col}_mean_rel'] = 0

            data_line[f'{col}_std'] = data_window[col].std()

            try:
                data_line[f'{col}_std_rel'] = data_window[col].std()/ data_window[col].iloc[-1]
            except:
                data_line[f'{col}_std_rel'] = 0

            try:
                data_line[f'{col}_spread'] = (data_window[col].max() - data_window[col].min()) / data_window[col].mean()
            except:
                data_line[f'{col}_spread'] = 0

        for extra_col in extra_cols:
            data_line[extra_col] = data_window[extra_col].iloc[0]


        data_line_full = pd.concat([data_line_full, pd.DataFrame(data_line, index=[0])])
    
    statistic_cols = list(data_line_full.columns)
    extra_cols = list(data.columns[~np.isin(data.columns, stock_info_cols + cols)])

    data_line_full[stock_info_cols] = data[stock_info].iloc[0]
    data_line_full = data_line_full[stock_info_cols + statistic_cols]

    return data_line_full
        


In [250]:
get_variations(data, windows, stock_info).head()

,Papel,Tipo,Empresa,Setor,Subsetor,cotacao,cotacao_rel,cotacao_median,cotacao_median_rel,cotacao_mean,...,evebit,mrgebit,mrgliq,roic,liqc,liq2m,c5y,year,date,Data_ult_cot_year
0,ALPA4,PN N1,ALPARGATAS PN N1,"Tecidos, Vestuário e Calçados",Calçados,15.15,0.130603,32.775,0.282543,54.016667,...,17.25,0.1121,0.0955,0.1895,2.30,5452290.0,0.1660,2013,2013-01-05,2023
0,ALPA4,PN N1,ALPARGATAS PN N1,"Tecidos, Vestuário e Calçados",Calçados,13.24,0.262698,14.195,0.281647,36.890000,...,12.72,0.1297,0.0920,0.1976,2.26,4604310.0,0.1312,2014,2014-01-08,2023
0,ALPA4,PN N1,ALPARGATAS PN N1,"Tecidos, Vestuário e Calçados",Calçados,7.07,0.059412,12.670,0.106471,29.668333,...,8.70,0.1013,0.0741,0.1605,2.28,5280290.0,0.1366,2015,2015-01-19,2023
0,ALPA4,PN N1,ALPARGATAS PN N1,"Tecidos, Vestuário e Calçados",Calçados,6.61,0.577293,11.775,1.028384,10.936667,...,6.91,0.1102,0.0734,0.1533,1.71,6008260.0,0.1327,2016,2016-01-06,2023
0,ALPA4,PN N1,ALPARGATAS PN N1,"Tecidos, Vestuário e Calçados",Calçados,12.80,1.057851,12.450,1.028926,11.161667,...,10.99,0.1372,0.0884,0.1950,2.27,7423590.0,0.0815,2017,2017-05-05,2023


In [251]:
def get_full_variations(data, tickers, window, stock_info_cols):

    df_variations = pd.DataFrame()

    for ticker in tickers:

        data_ticker = data[data['Papel']==ticker].sort_values('year', ascending=False)

        max = data_ticker['year'].max()
        min = data_ticker['year'].min()
        windows = get_windows(max, min, window=window, step=1)

        print(ticker)

        ticker_variation = get_variations(data_ticker, windows, stock_info_cols)

        df_variations = pd.concat([df_variations, ticker_variation])

    return df_variations


In [258]:
base_dataset = get_full_variations(df_actual, tickers, window=5, stock_info_cols=stock_info)

AGRO3
ALPA3


C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:20: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:27: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_median_rel'] = data_window[col].median()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:34: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_mean_rel'] = data_window[col].mean()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:41: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_std_rel'] = data_window[col].std()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  data_

ALPA4
AZEV3


C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:27: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_median_rel'] = data_window[col].median()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:34: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_mean_rel'] = data_window[col].mean()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:41: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_std_rel'] = data_window[col].std()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:20: RuntimeWarning: invalid value encountered in double_scalars
  data_

AZEV4
BAHI3


...

AFLT3
ALUP4
ATOM3
CRPG5
CRPG6
EGIE3
ENGI11
FRTA3
MOVI3
PARD3
RAIL3
RNEW3
RNEW4


C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:20: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:27: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_median_rel'] = data_window[col].median()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:34: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_mean_rel'] = data_window[col].mean()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:41: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_std_rel'] = data_window[col].std()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:20: RuntimeWarning: invalid value encountered in double_scalars
  data_lin

STBP3
AZUL4
CAML3
CRFB3
DMMO3
IRBR3
SUZB3
WIZS3
WLMM4
B3SA3
HAPV3


C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:27: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_median_rel'] = data_window[col].median()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:34: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_mean_rel'] = data_window[col].mean()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:41: RuntimeWarning: divide by zero encountered in double_scalars
  data_line[f'{col}_std_rel'] = data_window[col].std()/ data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_10324\3115813170.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  dat

SAPR11
ALSO3
APER3
CEAB3
COGN3
ENAT3
LOGG3
NEOE3
NTCO3
SQIA3
TAEE3
TAEE4
TASA3
TASA4
VIVA3
YDUQ3
AERI3
ALPK3
AMBP3
ATMP3
AURA33
AVLL3
BOAS3
CASH3
CURY3
DMVF3
ENJU3
EQMA3B
EQPA3
EQPA5
ESPA3
GMAT3
GPAR3
HBRE3
HBSA3
INTB3
LAVV3
LJQQ3
LWSA3
MBLY3
MDNE3
MELK3
MTRE3
NGRD3
PDTC3
PETZ3
PGMN3
PLPL3
PRNR3
RDOR3
RRRP3
SEQL3
SIMH3
SOMA3
TFCO4
TIMS3
VAMO3
AESB3
AGXY3
ALLD3
AMER3
ARML3
ASAI3
BLAU3
BMOB3
BRBI11
BRIT3
CBAV3
CLSA3
CMIN3
CPLE11
CSED3
CXSE3
DESK3
DEXP3
DEXP4
DOTZ3
DXCO3
ELMD3
EPAR3
FIQE3
GGPS3
IFCM3
IGTI11
IGTI3
JALL3
KRSA3
LVTC3
MATD3
MEGA3
MLAS3
NINJ3
ONCO3
OPCT3
ORVR3
PORT3
RAIZ4
RCSL3
RECV3
SBFG3
SMFT3
SOJA3
SYNE3
TRAD3
TTEN3
VBBR3
VIIA3
VITT3
VVEO3
WEST3
AURE3
CSUD3
NEXP3
VSTE3
WIZC3
ZAMP3


In [259]:
base_dataset.head()

,Papel,Tipo,Empresa,Setor,Subsetor,cotacao,cotacao_rel,cotacao_median,cotacao_median_rel,cotacao_mean,...,evebit,mrgebit,mrgliq,roic,liqc,liq2m,c5y,year,date,Data_ult_cot_year
0,AGRO3,ON NM,BRASILAGRO ON NM,Agropecuária,Agricultura,10.09,0.840833,10.295,0.857917,9.825000,...,-30.41,-0.1345,-0.1756,-0.0327,2.16,365541.0,0.7445,2013.0,2013-01-05,2023.0
0,AGRO3,ON NM,BRASILAGRO ON NM,Agropecuária,Agricultura,9.39,1.565000,9.920,1.653333,9.390000,...,24.38,0.0912,0.1204,0.0367,2.38,182408.0,0.6863,2014.0,2014-01-08,2023.0
0,AGRO3,ON NM,BRASILAGRO ON NM,Agropecuária,Agricultura,8.50,0.809524,9.920,0.944762,9.806667,...,-24.87,-0.1251,-0.0518,-0.0313,1.36,269495.0,0.4543,2015.0,2015-01-19,2023.0
0,AGRO3,ON NM,BRASILAGRO ON NM,Agropecuária,Agricultura,10.99,1.035815,9.920,0.934967,9.888333,...,2.51,0.4425,0.5806,0.2451,3.34,899610.0,0.2817,2016.0,2016-01-06,2023.0
0,AGRO3,ON NM,BRASILAGRO ON NM,Agropecuária,Agricultura,12.80,1.312821,9.920,1.017436,10.253333,...,-18.94,-0.3629,-0.3303,-0.0572,2.06,1057180.0,0.0139,2017.0,2017-05-05,2023.0


### Storing refined data into Google Cloud

In [263]:
client, bucket = initialize_bucket(credentials_path)

# upload base dataset to Google Cloud Storage
blob = bucket.blob('refined/base_dataset.csv')
blob.upload_from_string(base_dataset.to_csv(), 'text/csv',)